## Face Recognition – Unlock Your Computer With Your Face!


### مرحله ۱ - ایجاد داده‌های آموزشی

In [ ]:
import cv2
import numpy as np

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './faces/user/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

این کد برای جمع‌آوری داده‌های تصویری از چهره شما با استفاده از وب‌کم و ذخیره آن‌ها برای آموزش مدل تشخیص چهره استفاده می‌شود. توضیح بخش‌های مختلف کد به شرح زیر است:

- ابتدا کتابخانه‌های مورد نیاز (`cv2` و `numpy`) وارد می‌شوند.
- یک طبقه‌بند (classifier) تشخیص چهره با استفاده از فایل Haar Cascade بارگذاری می‌شود.
- تابع `face_extractor` تعریف شده تا چهره را در تصویر پیدا کند و قسمت چهره را برش دهد. اگر چهره‌ای پیدا نشود، مقدار `None` برمی‌گرداند.
- وب‌کم فعال می‌شود و یک شمارنده برای تعداد تصاویر جمع‌آوری‌شده تعریف می‌شود.
- در یک حلقه بی‌نهایت، هر بار یک فریم از وب‌کم خوانده می‌شود. اگر چهره‌ای در تصویر پیدا شود:
    - شمارنده یکی افزایش می‌یابد.
    - چهره برش داده شده به اندازه ۲۰۰x۲۰۰ تغییر اندازه داده و به تصویر خاکستری تبدیل می‌شود.
    - تصویر چهره در مسیر مشخص‌شده ذخیره می‌شود.
    - شماره نمونه روی تصویر نمایش داده می‌شود و تصویر نمایش داده می‌شود.
- اگر چهره‌ای پیدا نشود، پیام "Face not found" چاپ می‌شود.
- حلقه زمانی متوقف می‌شود که کلید Enter فشرده شود یا ۱۰۰ تصویر جمع‌آوری شود.
- در پایان، وب‌کم و پنجره‌های باز شده بسته می‌شوند و پیام اتمام جمع‌آوری نمونه‌ها نمایش داده می‌شود.


### مرحله ۲ - آموزش مدل

In [2]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Get the training data we previously made
data_path = './faces/user/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
model = cv2.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()

# Let's train our model 
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")


Model trained sucessefully


این کد وظیفه آموزش مدل تشخیص چهره را با استفاده از تصاویر جمع‌آوری‌شده بر عهده دارد. شرح گام‌های انجام‌شده در کد به صورت زیر است:

- ابتدا کتابخانه‌های مورد نیاز (`cv2`، `numpy` و توابع مربوط به مدیریت فایل‌ها) وارد می‌شوند.
- مسیر پوشه‌ای که تصاویر چهره در آن ذخیره شده‌اند تعیین می‌شود و نام تمام فایل‌های موجود در آن پوشه خوانده می‌شود.
- دو لیست برای داده‌های آموزشی (`Training_Data`) و برچسب‌ها (`Labels`) ایجاد می‌شود.
- با استفاده از یک حلقه، هر تصویر به صورت سیاه‌وسفید خوانده شده و به لیست داده‌های آموزشی افزوده می‌شود. همچنین برای هر تصویر یک برچسب عددی در لیست برچسب‌ها ذخیره می‌شود.
- لیست برچسب‌ها به آرایه‌ی numpy تبدیل می‌شود تا برای آموزش مدل قابل استفاده باشد.
- یک مدل تشخیص چهره با استفاده از الگوریتم LBPH ساخته می‌شود.
- مدل با داده‌های آموزشی و برچسب‌ها آموزش داده می‌شود.
- در پایان، پیام موفقیت‌آمیز بودن آموزش مدل نمایش داده می‌شود.

این مرحله باعث می‌شود مدل بتواند ویژگی‌های چهره شما را یاد بگیرد و در مراحل بعدی برای شناسایی چهره مورد استفاده قرار گیرد.

### مرحله ۳ - اجرای تشخیص چهره

In [7]:
import cv2
import numpy as np


face_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 75:
            cv2.putText(image, "Unlocked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
        else:
            cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()     

این کد مرحله نهایی سیستم تشخیص چهره و باز کردن قفل (Unlock) را اجرا می‌کند. شرح بخش‌های مختلف کد به صورت زیر است:

- کتابخانه‌های مورد نیاز (`cv2` و `numpy`) وارد می‌شوند.
- طبقه‌بند تشخیص چهره با استفاده از فایل Haar Cascade بارگذاری می‌شود.
- تابع `face_detector` برای شناسایی چهره در تصویر ورودی تعریف شده است. این تابع چهره را پیدا کرده، آن را برش می‌دهد و مستطیلی دور آن رسم می‌کند. اگر چهره‌ای شناسایی نشود، تصویر اصلی و یک لیست خالی بازمی‌گرداند.
- وب‌کم فعال می‌شود و در یک حلقه بی‌نهایت، هر بار یک فریم از وب‌کم دریافت می‌شود.
- با استفاده از تابع `face_detector`، چهره در تصویر شناسایی و استخراج می‌شود.
- اگر چهره‌ای شناسایی شود، تصویر چهره به حالت خاکستری تبدیل شده و به مدل آموزش‌دیده داده می‌شود تا پیش‌بینی کند که آیا چهره متعلق به کاربر است یا خیر.
- مقدار اطمینان (confidence) محاسبه و روی تصویر نمایش داده می‌شود.
- اگر اطمینان بیشتر از ۷۵٪ باشد، پیام "Unlocked" (قفل باز شد) نمایش داده می‌شود؛ در غیر این صورت، پیام "Locked" (قفل بسته است) نمایش داده می‌شود.
- اگر چهره‌ای شناسایی نشود، پیام‌های "No Face Found" و "Locked" نمایش داده می‌شوند.
- حلقه با فشردن کلید Enter متوقف می‌شود و در پایان، وب‌کم و تمام پنجره‌های باز شده بسته می‌شوند.

این کد با استفاده از مدل آموزش‌دیده، چهره کاربر را به صورت زنده شناسایی می‌کند و در صورت تطابق، قفل سیستم را باز می‌نماید. این فرآیند نمونه‌ای کاربردی از سیستم‌های احراز هویت مبتنی بر تشخیص چهره است.